In [1]:
import http.client
import json
from tqdm import tqdm
import pandas as pd

## Connection
### Setup the connection

Let's open the connection for the api

In [79]:
CONN = http.client.HTTPSConnection("streeteasy-api.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "6e5a5b57c4mshe184063a106d6c9p1e8fc6jsn06f32cfb4168",
    'x-rapidapi-host': "streeteasy-api.p.rapidapi.com"
}

### Function to get the request data 

In [54]:
def get_request(url:str, headers: dict[str, str], connection_obj:http.client.HTTPSConnection, method:str ='GET') -> str:
    connection_obj.request(method, url, headers=headers)
    res = connection_obj.getresponse()
    data = res.read()
    return data.decode("utf-8")

In [55]:
manhatttan_areas_supported = {
    'roosevelt-island', 'civic-center', 'fultonseaport', 'financial-district', 'tribeca', 
    'stuyvesant-townpcv', 'soho', 'hudson-square', 'little-italy', 'lower-east-side', 
    'two-bridges', 'chinatown', 'battery-park-city', 'gramercy-park', 'chelsea', 
    'west-chelsea', 'greenwich-village', 'noho', 'east-village', 'west-village', 
    'flatiron', 'nomad', 'nolita', 'midtown', 'central-park-south', 
    'midtown-south', 'midtown-east', 'murray-hill', 'sutton-place', 'turtle-bay', 
    'beekman', 'kips-bay', 'midtown-west', 'hudson-yards', 'hells-kitchen', 
    'upper-west-side', 'lincoln-square', 'manhattan-valley', 'upper-east-side', 'lenox-hill', 
    'yorkville', 'carnegie-hill', 'upper-carnegie-hill', 'morningside-heights', 'hamilton-heights',
    'washington-heights', 'hudson-heights', 'fort-george', 'inwood', 'west-harlem',
    'manhattanville', 'central-harlem', 'south-harlem', 'east-harlem', 'marble-hill'}
brooklyn_areas_supported = {
    'bath-beach', 'bay-ridge', 'bedford-stuyvesant', 'bensonhurst', 'bergen-beach',
    'boerum-hill', 'borough-park', 'brighton-beach', 'brooklyn-heights', 'brownsville',
    'bushwick', 'canarsie', 'carroll-gardens', 'city-line', 'clinton-hill', 
    'cobble-hill', 'columbia-st-waterfront-district', 'coney-island', 'crown-heights', 
    'cypress-hills', 'ditmas-park', 'downtown-brooklyn', 'dumbo', 'dyker-heights', 
    'east-flatbush', 'east-new-york', 'east-williamsburg', 'farragut', 'fiske-terrace', 
    'flatbush', 'flatlands', 'fort-greene', 'fort-hamilton', 'gerritsen-beach', 
    'gowanus', 'gravesend', 'greenpoint', 'greenwood', 'homecrest', 'kensington', 
    'madison', 'manhattan-beach', 'mapleton', 'marine-park', 'midwood', 'mill-basin', 
    'new-lots', 'ocean-hill', 'ocean-parkway', 'old-mill-basin', 'park-slope', 
    'prospect-heights', 'prospect-lefferts-gardens', 'prospect-park-south', 'red-hook', 
    'seagate', 'sheepshead-bay', 'starrett-city', 'stuyvesant-heights', 'sunset-park', 
    'vinegar-hill', 'weeksville', 'williamsburg', 'windsor-terrace', 'wingate'
}
queens_areas_supported = {
    'astoria', 'auburndale', 'bay-terrace-queens', 'bayside', 'beechhurst',
    'college-point', 'corona', 'ditmars-steinway', 'douglaston', 'east-elmhurst',
    'east-flushing', 'elmhurst', 'flushing', 'forest-hills', 'fresh-meadows',
    'glendale', 'hamilton-beach', 'hollis', 'howard-beach', 'hunters-point',
    'jackson-heights', 'jamaica', 'jamaica-hills', 'kew-gardens', 'kew-gardens-hills',
    'lindenwood', 'little-neck', 'long-island-city', 'malba', 'maspeth',
    'middle-village', 'murray-hill-queens', 'north-corona', 'old-howard-beach',
    'ozone-park', 'ramblersville', 'rego-park', 'richmond-hill', 'ridgewood',
    'rockwood-park', 'south-jamaica', 'south-ozone-park', 'st-albans',
    'sunnyside', 'whitestone', 'woodhaven', 'woodside', 'brookville',
    'laurelton', 'cambria-heights', 'queens-village', 'glen-oaks', 'floral-park',
    'bellerose', 'rosedale', 'springfield-gardens', 'briarwood', 'jamaica-estates',
    'new-hyde-park', 'south-richmond-hill', 'oakland-gardens', 'hillcrest', 
    'pomonok', 'utopia', 'clearview', 'rockaway-all', 'far-rockaway', 
    'broad-channel', 'arverne', 'rockaway-park', 'bayswater', 'belle-harbor', 
    'breezy-point', 'neponsit', 'edgemere', 'hammels'
}
bronx_supported_areas = {
    'mott-haven', 'north-new-york', 'melrose', 'port-morris', 'hunts-point',
    'longwood', 'morrisania', 'claremont', 'crotona-park-east', 'highbridge',
    'concourse', 'morris-heights', 'university-heights', 'fordham', 'east-tremont',
    'west-farms', 'belmont', 'bedford-park', 'kingsbridge', 'kingsbridge-heights',
    'riverdale', 'fieldston', 'spuyten-duyvil', 'soundview', 'castle-hill',
    'parkchester', 'throgs-neck', 'locust-point', 'pelham-bay', 'co-op-city',
    'city-island', 'morris-park', 'pelham-parkway', 'van-nest', 'laconia',
    'williamsbridge', 'baychester', 'woodlawn', 'wakefield', 'eastchester',
    'tremont', 'mt-hope', 'norwood', 'bronxwood', 'pelham-gardens', 'woodstock',
    'westchester-village', 'westchester-square', 'country-club',
    'schuylerville', 'edenwald'
}
staten_island_supported_areas = {
    'north-shore', 'arlington', 'clifton', 'elm-park', 'grymes-hill', 'howland-hook',
    'mariners-harbor', 'new-brighton', 'park-hill', 'port-richmond', 'rosebank', 
    'shore-acres', 'silver-lake', 'saint-george', 'stapleton', 'tompkinsville', 
    'west-brighton', 'south-shore', 'annadale', 'arden-heights', 'charleston', 
    'eltingville', 'great-kills', 'greenridge', 'huguenot', 'pleasant-plains', 
    'princes-bay', 'richmond-valley', 'rossville', 'tottenville', 'woodrow', 
    'east-shore', 'arrochar', 'bay-terrace', 'dongan-hills', 'egbertville', 
    'emerson-hill', 'fort-wadsworth', 'grant-city', 'grasmere', 'lighthouse-hill', 
    'midland-beach', 'new-dorp', 'oakwood', 'ocean-breeze', 'richmondtown', 
    'south-beach', 'todt-hill', 'new-dorp-beach', 'oakwood-beach', 'west-shore', 
    'bloomfield', 'chelsea-staten-island', 'travis', 'mid-island', 'willowbrook'
    'bulls-head', 'castleton-corners', 'graniteville', 'manor-heights', 
    'meiers-corners', 'new-springville', 'sunnyside-staten-island', 
    'westerleigh'
}

areas_supported = manhatttan_areas_supported.union(brooklyn_areas_supported).union(queens_areas_supported).union(bronx_supported_areas).union(staten_island_supported_areas)

### Time to loop over all supported areas

In [ ]:
total_area_listings = []
broken_stuff:list[tuple[str, str]] = []
progress = tqdm(areas_supported, desc="Processing Areas")
for supported_area in progress:
    area_listings = get_request(f"/rentals/search?areas={supported_area}", headers, CONN)
    try:
        area_listings = json.loads(area_listings)
    except json.JSONDecodeError:
        broken_stuff.append((supported_area, area_listings))
    else:
        total_area_listings.extend(area_listings['listings'])
    finally:
        progress.set_postfix({"Listings Accumulated": len(total_area_listings)})

Processing Areas: 100%|██████████| 310/310 [02:06<00:00,  2.45it/s, Listings Accumulated=4100]


In [ ]:
print(f"We found about {len(total_area_listings)} listings")
print("Following things broke:")
for idx, (area, error) in enumerate(broken_stuff):
    print(f"{idx+1}. Area: {area} Error: {error}")

We found about 4100 listings
Following things broke:
1. Area: willowbrookbulls-head Error: Found unknown areas: willowbrookbulls-head


## Fetching Data


In [ ]:
total_house_listings = []
other_broken_listings = []
progress = tqdm(total_area_listings, desc="Processing Areas")
for area_listing in progress:
    listing_data = get_request(f"/rentals/{area_listing['id']}", headers, CONN)
    try:
        listing_data = json.loads(listing_data)
    except json.JSONDecodeError:
        other_broken_listings.append((area_listing['id'], listing_data))
    else:
        total_house_listings.append(listing_data)
    finally:
        progress.set_postfix({"Listings Accumulated": len(total_house_listings)})
CONN.close()


Processing Areas: 100%|██████████| 4100/4100 [13:01<00:00,  5.25it/s, Listings Accumulated=4100]


In [81]:
print(f"We found about {len(total_house_listings)} listings")
print("Following things broke:")
for idx, (area, error) in enumerate(other_broken_listings):
    print(f"{idx+1}. Area: {area} Error: {error}")

We found about 4100 listings
Following things broke:


## Let's make a copy

In [ ]:
df = pd.DataFrame(total_house_listings)
df.to_csv("streeteasy_rentals.csv", index=False)

df = pd.DataFrame(total_area_listings)
df.to_csv("streeteasy_rental_ids.csv", index=False)

In [87]:
all_house_listings = pd.DataFrame(total_house_listings)
all_house_listings.head()

,id,listedAt,daysOnMarket,availableFrom,address,price,borough,neighborhood,zipcode,propertyType,...,longitude,amenities,builtIn,description,building,agents,noFee,images,videos,floorplans
0,4597458,2024-11-25,1,2024-11-25,171 East 96th Street #4A,1855,brooklyn,brownsville,11212,rental,...,-73.922293,"[fios_available, hardwood_floors, nyc_evacuati...",1926,Large newly renovated 1-bedroom \n\nFeatures: ...,{'id': '166259'},[Voro Purple LLC],False,[https://photos.zillowstatic.com/fp/9deccb9a3c...,[],[]
1,4596229,2024-11-22,4,2024-11-22,93 East 96th Street #11,2235,brooklyn,brownsville,11212,rental,...,-73.924122,"[fios_available, hardwood_floors, live_in_supe...",1926,Beautifully newly Renovated Two-Bedroom Apartm...,{'id': '165748'},[Kevin Cameron],False,[https://photos.zillowstatic.com/fp/49380b70ae...,[],[]
2,4596223,2024-11-22,4,2024-11-22,501 Saratoga Avenue #B,1750,brooklyn,brownsville,11212,rental,...,-73.917055,"[fios_available, hardwood_floors, live_in_supe...",1930,Stunning Newly Renovated One-Bedroom Apartment...,{'id': '140629'},[Kevin Cameron],False,[https://photos.zillowstatic.com/fp/e941cdddcb...,[],[]
3,4596221,2024-11-22,4,2024-11-22,501 Saratoga Avenue #C,2000,brooklyn,brownsville,11212,rental,...,-73.917055,"[fios_available, hardwood_floors, live_in_supe...",1930,Stunning newly renovated two-bedroom apt avail...,{'id': '140629'},[Kevin Cameron],False,[https://photos.zillowstatic.com/fp/8189b5ea3d...,[],[]
4,4594153,2024-11-20,6,2024-11-20,1115 Willmohr Street #2P,1850,brooklyn,brownsville,11212,rental,...,-73.917750,"[cats, doorman, elevator, fios_available, hard...",1961,BIG 1 Bedroom LUXURY APARTMENT\n\nThis 1 bathr...,{'id': '167432'},[Eli Loebenstein],False,[https://photos.zillowstatic.com/fp/f7a8a2c7d4...,[],[]


In [2]:
df = pd.read_csv("streeteasy_listings.csv")

In [ ]:
df.head()

In [ ]:
import usaddress
import string
from Levenshtein import distance as levenshtein_distance

In [ ]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

directional_map = {
    'n': 'north', 'north': 'north',
    's': 'south', 'south': 'south',
    'e': 'east', 'east': 'east',
    'w': 'west', 'west': 'west',
    'ne': 'northeast', 'northeast': 'northeast',
    'nw': 'northwest', 'northwest': 'northwest',
    'se': 'southeast', 'southeast': 'southeast',
    'sw': 'southwest', 'southwest': 'southwest'
}
suffix_map = {
    'st': 'street', 'street': 'street',
    'ave': 'avenue', 'avenue': 'avenue',
    'blvd': 'boulevard', 'boulevard': 'boulevard',
    'rd': 'road', 'road': 'road',
    'dr': 'drive', 'drive': 'drive',
    'ct': 'court', 'court': 'court',
    'pl': 'place', 'place': 'place',
    'ln': 'lane', 'lane': 'lane',
    'ter': 'terrace', 'terrace': 'terrace'
}

def normalize_street_name(street_name):
    # Split the street name into parts
    parts = street_name.lower().split()
    normalized_parts = []

    for part in parts:
        # Remove punctuation
        part_clean = remove_punctuation(part)
        # Normalize directionals and suffixes
        if part_clean in directional_map:
            normalized_parts.append(directional_map[part_clean])
        elif part_clean in suffix_map:
            normalized_parts.append(suffix_map[part_clean])
        else:
            normalized_parts.append(part_clean)
    return ' '.join(normalized_parts)

def normalize_with_usaddress(address):
    try:
        # Clean the address before parsing
        clean_address = remove_punctuation(address)
        parsed = usaddress.tag(clean_address)[0]
        
        # Normalize StreetNamePreDirectional
        if 'StreetNamePreDirectional' in parsed:
            value = parsed['StreetNamePreDirectional'].lower()
            parsed['StreetNamePreDirectional'] = directional_map.get(value, value)
        
        # Normalize StreetNamePostType
        if 'StreetNamePostType' in parsed:
            value = parsed['StreetNamePostType'].lower()
            parsed['StreetNamePostType'] = suffix_map.get(value, value)
        
        # Normalize StreetName
        if 'StreetName' in parsed:
            parsed['StreetName'] = normalize_street_name(parsed['StreetName'])
        
        # Normalize OccupancyIdentifier
        occupancy_id = parsed.get('OccupancyIdentifier', '')
        if occupancy_id:
            parsed['OccupancyIdentifier'] = remove_punctuation(occupancy_id).lower()
        else:
            # Check for SubaddressIdentifier if OccupancyIdentifier is missing
            subaddress_id = parsed.get('SubaddressIdentifier', '')
            if subaddress_id:
                parsed['OccupancyIdentifier'] = remove_punctuation(subaddress_id).lower()
        
        # Reconstruct the normalized address without OccupancyType
        normalized_address = " ".join(filter(None, [
            parsed.get('AddressNumber', '').lower(),
            parsed.get('StreetNamePreDirectional', '').lower(),
            parsed.get('StreetName', '').lower(),
            parsed.get('StreetNamePostType', '').lower(),
            parsed.get('OccupancyIdentifier', '')
        ])).lower()
        
        return normalized_address
    
    except usaddress.RepeatedLabelError as e:
        # Log the error and return a cleaned, lowercased address
        print(f"Error parsing address '{address}': {e}")
        return remove_punctuation(address).lower()

def are_addresses_same(addr1, addr2, threshold=2):
    norm_addr1 = normalize_with_usaddress(addr1)
    norm_addr2 = normalize_with_usaddress(addr2)
    distance = levenshtein_distance(norm_addr1, norm_addr2)
    return distance <= threshold

# Example
address1 = "245 E 23rd St Apt #4A"
address2 = "245 East 23rd Street 4A"

norm_addr1 = normalize_with_usaddress(address1)
norm_addr2 = normalize_with_usaddress(address2)

print("Normalized Address 1:", norm_addr1)
print("Normalized Address 2:", norm_addr2)
print("Are addresses the same?", are_addresses_same(address1, address2))

Normalized Address 1: 245 east 23rd street 4a
Normalized Address 2: 245 east 23rd street 4a
Are addresses the same? True
